In [ ]:
# cmd shift 5 for area, this code for x_y
import pyautogui
import time

print("🖱️  Mouse position (1/sec) - Ctrl+C to quit")
try:
    while True:
        x, y = pyautogui.position()
        print(f"\rX: {x:4d} Y: {y:4d}", end='', flush=True)
        time.sleep(1)
except KeyboardInterrupt:
    print("\n✅ Stopped.")


🖱️  Mouse position (1/sec) - Ctrl+C to quit
X: 1147 Y:  326
✅ Stopped.


In [22]:
import pyautogui
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
from gradio_client import Client, handle_file
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime
import os

# CONFIGURATION
TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT = 50, 227, 283, 250
INTERVAL_SECONDS = 2.0
CSV_FILE = "fidelity_orderbook_log.csv"
CLEAN_CSV_FILE = "fidelity_orderbook_clean.csv"

# REUSE CLIENT (critical for speed)
client = Client("deepdoctection/deepdoctection")

print(f"🚀 Starting order book monitor (every {INTERVAL_SECONDS}s)")
print(f"Table region: ({TABLE_X}, {TABLE_Y}, {TABLE_WIDTH}, {TABLE_HEIGHT})")

iteration = 0
try:
    while True:
        start_time = time.time()
        iteration += 1

        try:
            # 1. CAPTURE SCREENSHOT
            screenshot = pyautogui.screenshot(region=(TABLE_X, TABLE_Y, TABLE_WIDTH, TABLE_HEIGHT))

            # 2. PREPROCESS IMAGE
            img = screenshot.convert("RGB")
            gray = ImageEnhance.Contrast(img.convert("L")).enhance(1.2)
            gray = gray.filter(ImageFilter.SHARPEN)
            inverted = ImageOps.invert(gray)
            binary = inverted.point(lambda p: 0 if p > 140 else 255)
            binary.save("processed_table_region.png")

            # 3. OCR VIA DEEPDOCTECTION
            result = client.predict(
                img=handle_file("processed_table_region.png"),
                pdf=None,
                max_datapoints=1,
                api_name="/analyze_image",
            )

            # 4. PARSE TABLE
            if isinstance(result, tuple) and len(result) >= 4:
                html_table = result[3]
                soup = BeautifulSoup(html_table, "html.parser")
                table = soup.find("table")
                df = pd.read_html(str(table))[0]

                if df.columns.isnull().any():
                    df.columns = [f"col_{i}" if pd.isna(c) else str(c).strip() for i, c in enumerate(df.columns)]
                else:
                    df.columns = [str(c).strip() for c in df.columns]

                flat_values = df.to_numpy().ravel()
                col_labels = []
                for i in range(len(df)):
                    for col in df.columns:
                        col_labels.append(f"{col}_{i+1}")

                if len(col_labels) != len(flat_values):
                    min_len = min(len(col_labels), len(flat_values))
                    col_labels = col_labels[:min_len]
                    flat_values = flat_values[:min_len]

                row_data = [datetime.now().strftime("%Y-%m-%d %H:%M:%S")] + list(flat_values)
                col_names = ["timestamp"] + col_labels

                single_row = pd.DataFrame([row_data], columns=col_names)

                header_needed = not os.path.exists(CSV_FILE)
                single_row.to_csv(CSV_FILE, index=False, mode="a", header=header_needed)

                print(f"[{iteration:3d}] {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


            else:
                print(f"[{iteration}]  No table detected")

        except Exception as e:
            print(f"[{iteration}]  Error: {e}")
            continue

        elapsed = time.time() - start_time
        sleep_time = max(0, INTERVAL_SECONDS - elapsed)
        if sleep_time > 0:
            time.sleep(sleep_time)


except KeyboardInterrupt:
    print("\n Stopped by user — starting CSV cleanup...")

# ===============================
# CLEANUP AND REFORMAT STORED CSV
# ===============================
if os.path.exists(CSV_FILE):
    df = pd.read_csv(CSV_FILE)

    BID_HEADERS = ['exch_b', 'size_b', 'bid', 'ask', 'size_a', 'exch_a']

    # Keep timestamp + data columns (skip first 6 junk header columns)
    data_columns = df.columns[6:]
    df_data = df[['timestamp'] + list(data_columns)].copy()

    # Drop first junk column and shift left by one
    first_junk_col = data_columns[0]
    df_data = df_data.drop(columns=[first_junk_col])

    # Create proper alternating column names
    col_names = ["timestamp"]
    level = 1
    for i in range(len(df_data.columns) - 1):  # Adjust for dropped column
        base_header = BID_HEADERS[i % 6]
        col_names.append(f"{base_header}_{level}")
        if (i + 1) % 6 == 0:
            level += 1

    df_data.columns = col_names
    df_data.to_csv(CLEAN_CSV_FILE, index=False)

    print(f"✅ Cleaned CSV saved → {CLEAN_CSV_FILE}")
    print("First 20 columns:")
    print(list(df_data.columns[:20]))
    print("\nSample preview:")
    print(df_data[['timestamp', 'exch_b_1', 'size_b_1', 'bid_1', 'ask_1', 'size_a_1', 'exch_a_1']].head(2))
else:
    print("⚠️ No CSV file found; skipping cleanup.")


Loaded as API: https://deepdoctection-deepdoctection.hf.space ✔
🚀 Starting order book monitor (every 2.0s)
Table region: (50, 227, 283, 250)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/745242070.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  1] 2025-12-06 09:48:15


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/745242070.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  2] 2025-12-06 09:48:25


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50643/745242070.py:53: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


[  3] 2025-12-06 09:48:34

 Stopped by user — starting CSV cleanup...
✅ Cleaned CSV saved → fidelity_orderbook_clean.csv
First 20 columns:
['timestamp', 'exch_b_1', 'size_b_1', 'bid_1', 'ask_1', 'size_a_1', 'exch_a_1', 'exch_b_2', 'size_b_2', 'bid_2', 'ask_2', 'size_a_2', 'exch_a_2', 'exch_b_3', 'size_b_3', 'bid_3', 'ask_3', 'size_a_3', 'exch_a_3', 'exch_b_4']

Sample preview:
             timestamp exch_b_1  size_b_1   bid_1   ask_1  size_a_1 exch_a_1
0  2025-12-06 09:48:15     ARCX         1  454.61  454.64       160     XNMS
1  2025-12-06 09:48:25     ARCX         1  454.61  454.64       160     XNMS


In [26]:
# Show ALL rows of df_data (temporarily override pandas display limits)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', None):
    print(df_data)

# Or save to file for easier viewing
df_data.to_csv("full_orderbook_view.csv", index=False)
print(f"\n💾 Full dataset also saved to full_orderbook_view.csv ({len(df_data)} rows, {len(df_data.columns)} columns)")


             timestamp exch_b_1  size_b_1   bid_1   ask_1  size_a_1 exch_a_1  \
0  2025-12-06 09:48:15     ARCX         1  454.61  454.64       160     XNMS   
1  2025-12-06 09:48:25     ARCX         1  454.61  454.64       160     XNMS   
2  2025-12-06 09:48:34     ARCX         1  454.61  454.64       160     XNMS   

  exch_b_2  size_b_2   bid_2   ask_2  size_a_2 exch_a_2 exch_b_3  size_b_3  \
0     XNMS        80  454.68  464.66      3884     ARCX     ARCX         3   
1     XNMS        80  454.68  464.66      3884     ARCX     ARCX         3   
2     XNMS        80  454.68  464.66      3884     ARCX     ARCX         3   

    bid_3   ask_3  size_a_3 exch_a_3 exch_b_4  size_b_4  bid_4  ask_4  \
0  454.55  454.69        80     EDGX     ARCX       700  454.5  454.7   
1  454.55  454.69        80     EDGX     ARCX       700  454.5  454.7   
2  454.55  454.69        80     EDGX     ARCX       700  454.5  454.7   

   size_a_4 exch_a_4 exch_b_5  size_b_5   bid_5   ask_5 size_a_5 exch_a_5